<a href="https://colab.research.google.com/github/Pakhi-Singha/Stock-Analyser/blob/main/Stock_Market_Analyzer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install streamlit
!pip install scikit-learn
!pip install matplotlib
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import streamlit as st
import matplotlib.pyplot as plt


# Function to download stock data
def get_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    if stock_data.empty:
        raise ValueError("No data fetched. Check the ticker symbol or date range.")

    stock_data.reset_index(inplace=True)
    stock_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    stock_data.set_index('Date', inplace=True)

    # Adding moving averages
    stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['SMA_200'] = stock_data['Close'].rolling(window=200).mean()

    # Avoid dropping critical rows
    stock_data.dropna(subset=['Close'], inplace=True)
    return stock_data

# Prepare data for prediction
def prepare_data(stock_data):
    stock_data['Close_Lag1'] = stock_data['Close'].shift(1)
    stock_data['Close_Lag2'] = stock_data['Close'].shift(2)
    stock_data['Volume_Normalized'] = (stock_data['Volume'] - stock_data['Volume'].mean()) / stock_data['Volume'].std()
    stock_data.dropna(inplace=True)
    X = stock_data[['Close_Lag1', 'Close_Lag2', 'SMA_50', 'SMA_200', 'Volume_Normalized']]
    y = stock_data['Close']
    return X, y

# Train a model on combined data
def train_general_model(data_list):
    combined_X, combined_y = pd.concat([data[0] for data in data_list]), pd.concat([data[1] for data in data_list])
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(combined_X, combined_y)
    return model

# Main Streamlit app
def main():
    st.title("Stock Price Prediction")

    # User inputs
    ticker = st.text_input("Enter Stock Ticker (e.g., AAPL)", "AAPL")
    start_date = st.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end_date = st.date_input("End Date", pd.to_datetime("2024-12-31"))

    # Download stock data
    try:
        stock_data = get_stock_data(ticker, start_date, end_date)
    except ValueError as e:
        st.error(str(e))
        return

    st.write("Stock Data", stock_data.tail())

    # Prepare the data for the selected stock
    X, y = prepare_data(stock_data)

    # Load data for multiple stocks (replace this with actual combined data)
    # Example: use tickers like ['AAPL', 'MSFT', 'GOOGL', ...]
    tickers = ['AAPL', 'MSFT']  # Add more tickers as needed
    data_list = []
    for t in tickers:
        try:
            temp_data = get_stock_data(t, start_date, end_date)
            data_list.append(prepare_data(temp_data))
        except Exception:
            continue

    # Train the general model
    model = train_general_model(data_list)

    # Predict prices for the selected stock
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Predict future prices
    last_features = X.iloc[-1].values
    forecast = []
    current_features = last_features.copy()
    for _ in range(12):  # Predict for 12 months
        pred = model.predict(current_features.reshape(1, -1))[0]
        forecast.append(pred)
        current_features = np.roll(current_features, -1)
        current_features[-1] = pred

    # Create future dates for monthly predictions
    future_dates = pd.date_range(stock_data.index[-1], periods=13, freq='M')[1:]
    forecast_df = pd.DataFrame({'Date': future_dates, 'Predicted Close': forecast})

    # Plot actual and predicted prices
    st.subheader("Stock Price Predictions (2024)")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(stock_data.index, stock_data['Close'], label='Actual Close Price', color='blue')
    ax.plot(forecast_df['Date'], forecast_df['Predicted Close'], label='Predicted Prices (2024)', color='orange', linestyle='dashed')
    ax.set_title(f'{ticker} Stock Price Predictions for 2024')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price (USD)')
    ax.legend()
    st.pyplot(fig)

    # Show forecasted data
    st.write("Future Predictions", forecast_df)

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!streamlit run app.py --server.port 8502 &> /dev/null &



In [ ]:
from pyngrok import ngrok

# Set your ngrok auth token if you haven't
ngrok.set_auth_token("2qEY92j83ZEb3IB7mdcRLfoCIAt_7h4ZtJZtFseSg45xv6x2u")

# Open a tunnel with the correct configuration
tunnel = ngrok.connect(addr="8502", proto="http")

# Print the public URL where the Streamlit app is exposed
public_url = tunnel.public_url
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: https://a0a9-34-30-232-172.ngrok-free.app


In [ ]:
!killall ngrok
